In [1]:
import codecs
import re

#### Below codes merge .props and .text into a .mg file

In [2]:
import codecs

f1 = codecs.open("test.text", 'r', 'utf-8')
f2 = codecs.open("test.props", 'r', 'utf-8')
sout = ''
s1 = f1.read().split('\n')
s2 = f2.read().split('\n')
f1.close()
f2.close()
for i in range(len(s1)):
    if s1[i] == '':
        sout += '\n'
        continue
    t1 = s1[i].split('\t')
    t2 = s2[i].split('\t')
    sout += ('\t'.join(t1 + t2) + '\n')

f = codecs.open("test.mg", 'w', 'utf-8')
f.write(sout)
f.close()


#### Below codes transform a conll style merged .props into a CRF++ style samples.

In [3]:
f=codecs.open("test.mg","r",'utf-8')
s=f.read()
s_chunks=[[r.strip().split('\t') for r in t.strip().split('\n')] for t in s.strip().split('\n\n')]

FileNotFoundError: [Errno 2] No such file or directory: 'trn.mg'

In [12]:
def fetch_label(s, current_label):
    ts=re.search(u'\((.*)\*(.*)\)', s)
    tb=re.search(u'\((.*)\*', s)
    te=re.search(u'\*(.*)\)', s)
    if ts is not None:
        return 'S-' + ts.group(1), 'O'
    if tb is not None:
        return 'B-' + tb.group(1), tb.group(1)
    if te is not None:
        return 'E-' + current_label, 'O'
    if current_label == 'O':
        return 'O', 'O'
    return 'I-' + current_label, current_label

In [15]:
samples = []
for i_sent, sent in enumerate(s_chunks):
    pos_verbs = []
    postag_verbs = []
    word_verbs = []
    for idx, m in enumerate(sent):
        if not m[2] == '-':
            pos_verbs += [idx]
            postag_verbs += [m[1]]
            word_verbs += [m[2]]
    n_verbs = len(pos_verbs)
    for i_verb in range(n_verbs):
        pos_relv = -pos_verbs[i_verb]
        sample_sent = []
        current_target = 'O'
        for s in sent:
            sample_token = s[0:2]
            sample_token += [postag_verbs[i_verb]]
            sample_token += [word_verbs[i_verb]]
            sample_token += [str(pos_relv)]
            pos_relv += 1
            return_label, current_target = fetch_label(s[3 + i_verb], current_target)
            sample_token += [return_label]
            sample_sent += [sample_token]
        samples += [sample_sent]
    

In [16]:
s_out = ''
for sent in samples:
    s_sent = []
    for token in sent:
        s_sent += ['\t'.join(token)]
    s_out += '\n'.join(s_sent)
    s_out += '\n\n'
f = codecs.open('trn2_withverb.samples','w','utf-8')
f.write(s_out)
f.close()

#### Below codes transform a .text file into a for-parser file

In [15]:
f=codecs.open("trn.text","r",'utf-8')
s=f.read()
s_chunks=[[r.strip().split('\t') for r in t.strip().split('\n')] for t in s.strip().split('\n\n')]

In [16]:
s_out = []
for sent in s_chunks:
    s_sent = []
    for tok in sent:
        s_sent += [tok[0]]
    s_out += [' '.join(s_sent)]
f = codecs.open("trn.forparser","w","utf-8")
f.write("\n".join(s_out))
f.close()

In [37]:
a='U0:%x[-2,1]/%x[2,1]'

In [42]:
t=re.findall('(%x\[(?P<row>[-]?\d),(?P<col>\d)\])',a)

In [43]:
t

[('%x[-2,1]', '-2', '1'), ('%x[2,1]', '2', '1')]

In [18]:
t.group(4)

IndexError: no such group